# **Step 1 :** Extracting relevant data for the prediction model.
We'll consider film features (e.g., title, release year, rating, length)
and rental features (e.g., rental rate, number of times rented, last rental date).










In [1]:
import mysql.connector
from mysql.connector import Error
import getpass

try:
    # Establishing the connection
    connection = mysql.connector.connect(
        host='localhost',
        port=3306,
        user='root',
        password = getpass.getpass("Enter password: "),
        database='sakila'
    )

    if connection.is_connected():
        db_info = connection.get_server_info()
        print("Successfully connected to MySQL Server version ", db_info)
        
        # Creating a cursor object using the cursor() method
        cursor = connection.cursor()
        
        # SQL query to fetch the necessary data
        query = """
        SELECT
            f.film_id,
            f.title,
            f.release_year,
            f.rating,
            f.length,
            f.rental_rate,
            COUNT(r.rental_id) AS number_of_rentals,
            MAX(r.rental_date) AS last_rental_date
        FROM
            film f
        LEFT JOIN inventory i ON f.film_id = i.film_id
        LEFT JOIN rental r ON i.inventory_id = r.inventory_id
        GROUP BY f.film_id

        """
        
        # Executing the SQL command
        cursor.execute(query)
        
        # Fetching all the rows
        records = cursor.fetchall()
        
        print("Data Retrieved: ")
        for row in records:
            print(row)
        
except Error as e:
    print("Error while connecting to MySQL", e)


Successfully connected to MySQL Server version  8.0.36
Data Retrieved: 
(1, 'ACADEMY DINOSAUR', 2006, 'PG', 86, Decimal('0.99'), 25, datetime.datetime(2024, 3, 12, 17, 17, 34))
(2, 'ACE GOLDFINGER', 2006, 'G', 48, Decimal('4.99'), 7, datetime.datetime(2006, 2, 14, 15, 16, 3))
(3, 'ADAPTATION HOLES', 2006, 'NC-17', 50, Decimal('2.99'), 12, datetime.datetime(2005, 8, 23, 13, 54, 39))
(4, 'AFFAIR PREJUDICE', 2006, 'G', 117, Decimal('2.99'), 23, datetime.datetime(2006, 2, 14, 15, 16, 3))
(5, 'AFRICAN EGG', 2006, 'G', 130, Decimal('2.99'), 12, datetime.datetime(2006, 2, 14, 15, 16, 3))
(6, 'AGENT TRUMAN', 2006, 'PG', 169, Decimal('2.99'), 21, datetime.datetime(2005, 8, 21, 16, 3, 1))
(7, 'AIRPLANE SIERRA', 2006, 'PG-13', 62, Decimal('4.99'), 15, datetime.datetime(2005, 8, 22, 17, 18, 5))
(8, 'AIRPORT POLLOCK', 2006, 'R', 54, Decimal('4.99'), 18, datetime.datetime(2005, 8, 23, 20, 24, 36))
(9, 'ALABAMA DEVIL', 2006, 'PG-13', 114, Decimal('2.99'), 12, datetime.datetime(2005, 8, 23, 14, 26, 51

# **Step 2 :** Reading the data into a Pandas DataFrame.

In [2]:
import pandas as pd


try:
    df = pd.read_sql_query(query, connection)
    display(df.head())
    
except Exception as e:
    print(f"Error: {e}")

C:\Users\wailb\AppData\Local\Temp\ipykernel_836\2346355635.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,film_id,title,release_year,rating,length,rental_rate,number_of_rentals,last_rental_date
0,1,ACADEMY DINOSAUR,2006,PG,86,0.99,25,2024-03-12 17:17:34
1,2,ACE GOLDFINGER,2006,G,48,4.99,7,2006-02-14 15:16:03
2,3,ADAPTATION HOLES,2006,NC-17,50,2.99,12,2005-08-23 13:54:39
3,4,AFFAIR PREJUDICE,2006,G,117,2.99,23,2006-02-14 15:16:03
4,5,AFRICAN EGG,2006,G,130,2.99,12,2006-02-14 15:16:03


# **Step 3 :** Analyzing extracted features and transforming them

In [3]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from datetime import datetime

# Before proceeding, let's make sure that the columns we intend to scale are purely numerical.
# This check helps avoid attempting to scale or transform non-numeric data.

numerical_features = ['length', 'rental_rate' ,'number_of_rentals']


# Checking if any of the specified columns contain non-numeric values
for column in numerical_features:
    if df[column].dtype != 'float64' and df[column].dtype != 'int64':
        print(f"Column {column} contains non-numeric data.")
    else:
        # Imputing missing values for numerical columns before scaling
        imputer = SimpleImputer(strategy='median')
        df[column] = imputer.fit_transform(df[[column]])


# Scaling Numerical Variables
scaler = StandardScaler()   
df[numerical_features] = scaler.fit_transform(df[numerical_features])

# Encoding Categorical Variables
encoder = OneHotEncoder(sparse_output=False)
rating_encoded = encoder.fit_transform(df[['rating']])
rating_columns = encoder.get_feature_names_out(['rating'])
df_encoded = pd.DataFrame(rating_encoded, columns=rating_columns)
df = pd.concat([df.drop('rating', axis=1), df_encoded], axis=1)


# Transforming Temporal Features
df['last_rental_date'] = pd.to_datetime(df['last_rental_date'], errors='coerce')
df['days_since_last_rental'] = (datetime.now() - df['last_rental_date']).dt.days
df.drop('last_rental_date', axis=1, inplace=True)


In [4]:
#checking for NaNs
df.isna().sum()

film_id                    0
title                      0
release_year               0
length                     0
rental_rate                0
number_of_rentals          0
rating_G                   0
rating_NC-17               0
rating_PG                  0
rating_PG-13               0
rating_R                   0
days_since_last_rental    42
dtype: int64

In [5]:
#Imputing missing values with median
median= df['days_since_last_rental'].median()

df['days_since_last_rental'].fillna(median, inplace=True)

In [6]:
# Showing the first few rows of the processed DataFrame to ensure transformations and cleaning were successful
display(df.head())
df.isna().sum()

,film_id,title,release_year,length,rental_rate,number_of_rentals,rating_G,rating_NC-17,rating_PG,rating_PG-13,rating_R,days_since_last_rental
0,1,ACADEMY DINOSAUR,2006,-0.724445,-1.209308,1.218802,0.0,0.0,1.0,0.0,0.0,23.0
1,2,ACE GOLDFINGER,2006,-1.664897,1.221461,-1.231325,1.0,0.0,0.0,0.0,0.0,6624.0
2,3,ADAPTATION HOLES,2006,-1.615399,0.006077,-0.550734,0.0,1.0,0.0,0.0,0.0,6799.0
3,4,AFFAIR PREJUDICE,2006,0.042766,0.006077,0.946566,1.0,0.0,0.0,0.0,0.0,6624.0
4,5,AFRICAN EGG,2006,0.364499,0.006077,-0.550734,1.0,0.0,0.0,0.0,0.0,6624.0


film_id                   0
title                     0
release_year              0
length                    0
rental_rate               0
number_of_rentals         0
rating_G                  0
rating_NC-17              0
rating_PG                 0
rating_PG-13              0
rating_R                  0
days_since_last_rental    0
dtype: int64

#  **Step 4 :** Creating a query to get the list of films and a boolean indicating if it was rented last month

In [7]:
# Calculating the most recent rental date in the dataset
most_recent_rental = df['days_since_last_rental'].min()

# Create a boolean column indicating if a film was rented last month
# Assuming "last month" refers to the past 30 days from the most recent rental
df['rented_last_month'] = df['days_since_last_rental'] <= 30

# Select the relevant columns for the query
query_result = df[['film_id', 'title', 'rented_last_month']]

query_result.head()


,film_id,title,rented_last_month
0,1,ACADEMY DINOSAUR,True
1,2,ACE GOLDFINGER,False
2,3,ADAPTATION HOLES,False
3,4,AFFAIR PREJUDICE,False
4,5,AFRICAN EGG,False


In [8]:
df.head()

,film_id,title,release_year,length,rental_rate,number_of_rentals,rating_G,rating_NC-17,rating_PG,rating_PG-13,rating_R,days_since_last_rental,rented_last_month
0,1,ACADEMY DINOSAUR,2006,-0.724445,-1.209308,1.218802,0.0,0.0,1.0,0.0,0.0,23.0,True
1,2,ACE GOLDFINGER,2006,-1.664897,1.221461,-1.231325,1.0,0.0,0.0,0.0,0.0,6624.0,False
2,3,ADAPTATION HOLES,2006,-1.615399,0.006077,-0.550734,0.0,1.0,0.0,0.0,0.0,6799.0,False
3,4,AFFAIR PREJUDICE,2006,0.042766,0.006077,0.946566,1.0,0.0,0.0,0.0,0.0,6624.0,False
4,5,AFRICAN EGG,2006,0.364499,0.006077,-0.550734,1.0,0.0,0.0,0.0,0.0,6624.0,False


# **Step 5** : Creating a logistic regression model to predict this variable from the cleaned data.

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.preprocessing import StandardScaler

# Preparing the dataset
X = df.drop(columns=['film_id', 'title', 'release_year', 'rented_last_month'])
y = df['rented_last_month']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Creating and fitting the logistic regression model
log_reg = LogisticRegression()
log_reg.fit(X_train_scaled, y_train)

# Predicting on the test set
y_pred = log_reg.predict(X_test_scaled)

# Evaluating the model
precision = precision_score(y_test, y_pred, zero_division=1)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

precision, recall, f1, accuracy

c:\Users\wailb\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\wailb\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(1.0, 0.0, 0.0, 1.0)

 ### __Insight:__
  The model perfectly predicts the majority class (not rented last month), reflected by 100% accuracy. 
  
  However, the lack of true positive predictions results in 0% recall and F1 score. 
  
  This outcome suggests the model is not effective for predicting the rare class (rented last month) in its current state, potentially due to __imbalance__ in the dataset